In [78]:
import numpy as np
from qiskit import(
  QuantumCircuit,
  execute,
  Aer)
from qiskit.visualization import plot_histogram

# Use Aer's qasm_simulator
simulator = Aer.get_backend('qasm_simulator')

In [79]:
n = 3 # state
num_qubits = n + 1


In [80]:
increment_circuit = QuantumCircuit(n, name='add1')
for i in range(n - 1):
    increment_circuit.mcx([j for j in range(n - 1 - i)], n - 1 - i)
increment_circuit.x(0)

# increment_circuit.draw()

increment_gate = increment_circuit.to_gate()
controlled_increment_gate = increment_gate.control()

In [81]:
decrement_circuit = QuantumCircuit(n, name='sub1')
decrement_circuit.x(0)
for i in range(n - 1):
    decrement_circuit.mcx([j for j in range(i+1)], i+1)

# decrement_circuit.draw()

decrement_circuit = decrement_circuit.to_gate()
controlled_decrement_gate = decrement_circuit.control()

In [134]:
num_steps = min(3, 2**n - 1)
# should only do at most 2^n - 1 steps (right bound for signed integer)
trials = 1000

qc = QuantumCircuit(num_qubits, n)

# first qubit is coin flip
# other qubits are state/position qubits, with one bit for sign (leftmost bit)

for i in range(num_steps):
    qc.h(0)
    # controlled increment
    qc.append(controlled_increment_gate, [0,1,2,3])
    
    
    # controlled decrement
    qc.x(0)
    qc.append(controlled_decrement_gate, [0,1,2,3])
    qc.x(0)
    
qc.draw()

# Map the quantum measurement to the classical bits
qc.measure([i+1 for i in range(n)], [i for i in range(n)])

# Execute the circuit on the qasm simulator
job = execute(qc, simulator, shots=trials)

# Grab results from the job
result = job.result()

# Returns counts
counts = result.get_counts(qc)
print("\nTotal counts (signed binary representation):", counts)

def to_decimal(num):
    if num[0]=='0':
        return int(num, 2)
    else:
        return int(num, 2) - (1 << len(num))

distribution = {}
for x in counts:
    distribution[to_decimal(x)] = counts[x]/trials

print("\nStates and Probabilities (decimal representation):", distribution)


Total counts (signed binary representation): {'001': 122, '011': 139, '101': 131, '111': 608}

States and Probabilities (decimal representation): {1: 0.125, 3: 0.118, -3: 0.108, -1: 0.649}
